# 1. Install and Import Dependencies

In [1]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
!pip install transformers

In [3]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 2. Setup Generator

In [4]:
summarizer = pipeline('summarization', model="t5-base", tokenizer="t5-base", framework="tf")

Downloading: 100%|██████████| 1.20k/1.20k [00:00<00:00, 307kB/s]


ValueError: Pipeline cannot infer suitable model classes from t5-base

# 3. Get Input Text

In [ ]:
URL = 'https://en.wikipedia.org/wiki/Harry_Potter'

In [ ]:
r = requests.get(URL)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])

In [ ]:
results

[<h1 class="firstHeading" id="firstHeading"><i>Harry Potter</i></h1>,
 <p class="mw-empty-elt">
 </p>,
 <p class="mw-empty-elt">
 </p>,
 <p><i><b>Harry Potter</b></i> is a series of seven <a href="/wiki/Fantasy_literature" title="Fantasy literature">fantasy novels</a> written by British author <a href="/wiki/J._K._Rowling" title="J. K. Rowling">J. K. Rowling</a>. The novels chronicle the lives of a young <a href="/wiki/Magician_(fantasy)" title="Magician (fantasy)">wizard</a>, <a href="/wiki/Harry_Potter_(character)" title="Harry Potter (character)">Harry Potter</a>, and his friends <a href="/wiki/Hermione_Granger" title="Hermione Granger">Hermione Granger</a> and <a href="/wiki/Ron_Weasley" title="Ron Weasley">Ron Weasley</a>, all of whom are students at <a class="mw-redirect" href="/wiki/Hogwarts_School_of_Witchcraft_and_Wizardry" title="Hogwarts School of Witchcraft and Wizardry">Hogwarts School of Witchcraft and Wizardry</a>. The main <a href="/wiki/Story_arc" title="Story arc">sto

In [ ]:
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
ARTICLE[0:100]

'Harry Potter \n \n Harry Potter is a series of seven fantasy novels written by British author J. K. Ro'

# 4. Chunk text

Append <eos> to punctuations that marks the end of a sentence. Without the <eos> tag, sentences will be split without any punctuation, which is weird.

In [ ]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
sentences = ARTICLE.split('<eos>')

In [ ]:
sentences[1]

' K.'

In [ ]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

0


In [ ]:
for chunk_id in range (len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [ ]:
len(chunks[0].split(' '))

476

# 4. Summarise Text

In [32]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)
